## Модель и квантование

* Base model: Qwen/Qwen2.5-7B-Instruct
* Загрузка: 4-bit через bitsandbytes
  * nf4
  * double quant = True
  * compute dtype: float16
  * Формат обучения: SFT (supervised fine-tuning) по messages → apply_chat_template → поле `text`

---

## LoRA-конфиг

* r = 16
* lora_alpha = 32
* lora_dropout = 0.05
* bias = "none"
* target_modules = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]

---

## Тренировка

* num_train_epochs = 1
* per_device_train_batch_size = 1
* gradient_accumulation_steps = 8  
  → эффективный batch ≈ 8 (по шагам)
* max_seq_length = 512
* packing = True  
  (важно: уплотняем несколько коротких примеров в один блок до 512)
* learning_rate = 1e-4
* scheduler: cosine
* warmup: 0.03
* optimizer: paged_adamw_8bit
* fp16 = True
* gradient_checkpointing = True
* max_grad_norm = 1.0
* логи: logging_steps = 20
* eval: eval_steps = 100
* save: save_steps = 100
* (если добавлял) neftune_noise_alpha = 5 — регуляризация против лупов

In [3]:
# загрузка файлов
from google.colab import files

uploaded = files.upload()

Saving train.jsonl to train.jsonl
Saving valid.jsonl to valid.jsonl


In [4]:
!ls -lh

total 8.0K
drwxr-xr-x 2 root root 4.0K Jan 19 10:05 data
drwxr-xr-x 1 root root 4.0K Dec  9 14:42 sample_data


In [3]:
!mkdir -p data
!mv train.jsonl valid.jsonl data/
!ls data

train.jsonl  valid.jsonl


In [5]:
# чекап
!nvidia-smi

Mon Jan 19 11:27:29 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   69C    P0             45W /   70W |    6420MiB /  15360MiB |     42%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
# чек библиотек
import numpy, pandas, pyarrow
import torch, transformers, datasets, bitsandbytes
print("numpy", numpy.__version__)
print("pandas", pandas.__version__)
print("pyarrow", pyarrow.__version__)
print("torch", torch.__version__)
print("transformers", transformers.__version__)
print("datasets", datasets.__version__)
print("bnb", bitsandbytes.__version__)

numpy 1.26.4
pandas 2.3.3
pyarrow 23.0.0
torch 2.3.1+cu121
transformers 4.43.3
datasets 2.20.0
bnb 0.43.1


In [4]:
!pip -q uninstall -y triton bitsandbytes
!pip -q install -U "triton==2.3.1"
!pip -q install -U "bitsandbytes==0.43.1"

In [4]:
!ls -lh
!head -n 1 train.jsonl
!head -n 1 valid.jsonl

total 2.1M
drwxr-xr-x 2 root root 4.0K Jan 19 10:05 data
drwxr-xr-x 1 root root 4.0K Dec  9 14:42 sample_data
-rw-r--r-- 1 root root 2.0M Jan 19 10:20 train.jsonl
-rw-r--r-- 1 root root 103K Jan 19 10:20 valid.jsonl
{"messages": [{"role": "system", "content": "Ты — поэтический генератор песен в стиле группы «Макулатура». Соблюдай структуру и теги секций (<VERSE>, <CHORUS>, <BRIDGE>, <INTRO>, <OUTRO>, <REFRAIN>, <HOOK>). Сохраняй голос, заданный атрибутом speaker (alekhin/speransky/group). Пиши по-русски. Не добавляй пояснений, только текст песен/секций в требуемом формате."}, {"role": "user", "content": "Продолжи песню в стиле группы «Макулатура».\nТема: поездка и расставание\nНиже приведён контекст (последние секции). Напиши СЛЕДУЮЩУЮ секцию полностью.\nВерни ровно один блок секции с тегами.\n\nКОНТЕКСТ:\n<VERSE index=1 speaker=alekhin>\nБессонница, Гомер\nи\nНед Фландерс\nРешая простейший вопрос, поругались\nМне было неплохо, когда мы расстались\nНо в руках все ломалось, во мне много

# 1) Подготовка датасета из messages

Мы берём каждую запись и превращаем в один текст через chat template токенизатора.

In [7]:
from datasets import load_dataset
from transformers import AutoTokenizer

MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True, trust_remote_code=True)

data_files = {"train": "train.jsonl", "validation": "valid.jsonl"}
ds = load_dataset("json", data_files=data_files)

def to_text(example):
    # example["messages"] = [{"role": "...", "content": "..."}, ...]
    text = tokenizer.apply_chat_template(
        example["messages"],
        tokenize=False,
        add_generation_prompt=False
    )
    return {"text": text}

ds = ds.map(to_text, remove_columns=ds["train"].column_names)
ds

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/442 [00:00<?, ? examples/s]

Map:   0%|          | 0/23 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 442
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 23
    })
})

In [8]:
print(ds["train"][0]["text"][:800])
print("\n---\n")
print(ds["train"][0]["text"][-800:])

<|im_start|>system
Ты — поэтический генератор песен в стиле группы «Макулатура». Соблюдай структуру и теги секций (<VERSE>, <CHORUS>, <BRIDGE>, <INTRO>, <OUTRO>, <REFRAIN>, <HOOK>). Сохраняй голос, заданный атрибутом speaker (alekhin/speransky/group). Пиши по-русски. Не добавляй пояснений, только текст песен/секций в требуемом формате.<|im_end|>
<|im_start|>user
Продолжи песню в стиле группы «Макулатура».
Тема: поездка и расставание
Ниже приведён контекст (последние секции). Напиши СЛЕДУЮЩУЮ секцию полностью.
Верни ровно один блок секции с тегами.

КОНТЕКСТ:
<VERSE index=1 speaker=alekhin>
Бессонница, Гомер
и
Нед Фландерс
Решая простейший вопрос, поругались
Мне было неплохо, когда мы расстались
Но в руках все ломалось, во мне много страсти
И я думал, мы будем вместе до старости
И я утешал 

---

 каждой гортани
Я вижу тебя вместо всех билетов в кино и клубы
Вижу твои глаза, растоптанные в городских лужах
Заворачиваю за угол и жду когда меня ножом задушат
С газетой "работа" под мышкой м

In [9]:
import re, random

def has_tags(t: str) -> bool:
    return bool(re.search(r"<(VERSE|CHORUS|OUTRO|BRIDGE|INTRO|REFRAIN|HOOK)\b", t))

samples = random.sample(range(len(ds["train"])), k=min(10, len(ds["train"])))
print("tagged in sample:", sum(has_tags(ds["train"][i]["text"]) for i in samples), "/", len(samples))

tagged in sample: 10 / 10


In [12]:
texts = ds["train"][:50]["text"]
lens = [len(tokenizer(t).input_ids) for t in texts]
print("tokens (first 50): min/avg/max =", min(lens), sum(lens)/len(lens), max(lens))

tokens (first 50): min/avg/max = 273 1004.12 1876


## 2) Задаём конфиг qLoRA + грузим модель в 4-bit

In [13]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,   # для T4 нормально
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True,
)

# Qwen2.5: эти модули обычно присутствуют и хорошо работают для стиля/языка
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj","k_proj","v_proj","o_proj",
        "gate_proj","up_proj","down_proj"
    ],
)

print("Loaded model.")

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

Loaded model.


## 3) Тренировка (TRL SFTTrainer)

In [14]:
from trl import SFTTrainer
from transformers import TrainingArguments

OUTPUT_DIR = "qwen_makulatura_lora"

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,

    evaluation_strategy="steps",
    eval_steps=100,
    save_steps=100,
    logging_steps=20,

    learning_rate=1e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,

    optim="paged_adamw_8bit",
    fp16=True,
    gradient_checkpointing=True,
    max_grad_norm=1.0,

    report_to="none",

    # 1) регуляризация, часто снижает лупы на малых датасетах
    neftune_noise_alpha=5,
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=ds["train"],
    eval_dataset=ds["validation"],
    peft_config=lora_config,
    dataset_text_field="text",

    max_seq_length=512,
    packing=True,

    # 2) чтобы trainer точно не пытался “прибивать” колонки
    dataset_kwargs={"add_special_tokens": False},

    args=training_args,
)

trainer.train()

/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length, packing, dataset_kwargs. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:192: UserWarning: You passed a `packing` argument to the SFTTrainer, the value yo

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
100,1.943000,1.997296


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=111, training_loss=2.1109014631391645, metrics={'train_runtime': 2467.9665, 'train_samples_per_second': 0.361, 'train_steps_per_second': 0.045, 'total_flos': 1.939832373982003e+16, 'train_loss': 2.1109014631391645, 'epoch': 0.9966329966329966})

## 4) Сохранить адаптер (LoRA)

Это уже готовый результат: папка qwen_makulatura_lora/

In [17]:
OUTPUT_DIR = "qwen_makulatura_lora"

trainer.model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

!ls -lah qwen_makulatura_lora

total 166M
drwxr-xr-x 4 root root 4.0K Jan 19 11:18 .
drwxr-xr-x 1 root root 4.0K Jan 19 10:36 ..
-rw-r--r-- 1 root root  727 Jan 19 11:20 adapter_config.json
-rw-r--r-- 1 root root 155M Jan 19 11:20 adapter_model.safetensors
-rw-r--r-- 1 root root  605 Jan 19 11:20 added_tokens.json
drwxr-xr-x 2 root root 4.0K Jan 19 11:13 checkpoint-100
drwxr-xr-x 2 root root 4.0K Jan 19 11:17 checkpoint-111
-rw-r--r-- 1 root root 1.6M Jan 19 11:20 merges.txt
-rw-r--r-- 1 root root 5.0K Jan 19 11:20 README.md
-rw-r--r-- 1 root root  613 Jan 19 11:20 special_tokens_map.json
-rw-r--r-- 1 root root 7.2K Jan 19 11:20 tokenizer_config.json
-rw-r--r-- 1 root root 6.8M Jan 19 11:20 tokenizer.json
-rw-r--r-- 1 root root 2.7M Jan 19 11:20 vocab.json


## 5) Генерация

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"
OUTPUT_DIR = "qwen_makulatura_lora"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

tok = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True, trust_remote_code=True)

base = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map={"": 0},          # принудительно на GPU без диска
    trust_remote_code=True,
)

model = PeftModel.from_pretrained(base, OUTPUT_DIR)

system = "Ты — поэтический генератор песен в стиле группы «Макулатура». Пиши по-русски. Верни ровно один блок секции с тегами."
user = """Тема: бывшая, одиночество, город
Сгенерируй РОВНО ОДНУ секцию:
<VERSE speaker=speransky>
...
</VERSE>
Минимум 10 строк. Без латиницы. Без повторов одной строки более 2 раз подряд.
"""

prompt = tok.apply_chat_template(
    [{"role":"system","content":system},{"role":"user","content":user}],
    tokenize=False,
    add_generation_prompt=True
)

inputs = tok(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    out = model.generate(
        **inputs,
        max_new_tokens=260,
        do_sample=True,
        temperature=0.75,
        top_p=0.9,
        top_k=60,
        repetition_penalty=1.08,
        eos_token_id=tok.eos_token_id,
    )

print(tok.decode(out[0], skip_special_tokens=True)[-2000:])

In [9]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"
OUTPUT_DIR = "qwen_makulatura_lora"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

tok = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True, trust_remote_code=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

base = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map={"": 0},
    trust_remote_code=True,
)

model = PeftModel.from_pretrained(base, OUTPUT_DIR)
model.eval()

system = "Ты — поэтический генератор песен в стиле группы «Макулатура». Пиши по-русски. Верни ровно один блок секции с тегами."

user = """Тема: бывшая, одиночество, город

Сгенерируй РОВНО ОДНУ секцию. Заполни секцию реальными строками — нельзя оставлять заглушки, троеточия или пустые места.

Формат (строго):
<VERSE speaker=speransky>
</VERSE>

Требования:
- минимум 10 строк ВНУТРИ секции
- без латиницы и мусорных символов
- не повторять одну и ту же строку/фразу более 2 раз подряд
- каждая строка: конкретный образ/действие/наблюдение (метро, подъезд, никотин, похмелье, холод, бессонница)
"""

prompt = tok.apply_chat_template(
    [{"role": "system", "content": system}, {"role": "user", "content": user}],
    tokenize=False,
    add_generation_prompt=True
)

inputs = tok(prompt, return_tensors="pt").to(model.device)

for seed in [42, 1337, 2024]:
    torch.manual_seed(seed)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=320,          # чуть больше, чтобы успела закрыть </VERSE>
            do_sample=True,
            temperature=0.78,            # чуть выше, чтобы не “залипала” на шаблоне
            top_p=0.9,
            top_k=60,
            repetition_penalty=1.12,     # сильнее режем лупы
            no_repeat_ngram_size=4,      # мягкий анти-повтор
            eos_token_id=tok.eos_token_id,
            pad_token_id=tok.pad_token_id,
            use_cache=True,
        )

    decoded = tok.decode(out[0], skip_special_tokens=True)
    completion = decoded[len(prompt):].lstrip() if decoded.startswith(prompt) else decoded

    print(f"\n--- SEED {seed} ---\n")
    print(completion[-2000:])

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

bad_words_ids: [[64476, 28, 4074], [64476, 28, 1574, 30664, 258]]

--- SEED 42 ---

system
Ты — поэтический генератор песен в стиле группы «Макулатура». Пиши по-русски. Верни ровно один блок секции с тегами.
user
Тема: бывшая, одиночество, город

Сгенерируй РОВНО ОДНУ секцию. Заполни секцию реальными строками — нельзя оставлять заглушки, троеточия или пустые места.
Важно: атрибут speaker должен быть строго speransky. Нельзя group и нельзя alekhin.

Формат (строго):
<VERSE speaker=speransky>
</VERSE>

Требования:
- минимум 10 строк ВНУТРИ секции
- без латиницы и мусорных символов
- не повторять одну и ту же строку/фразу более 2 раз подряд
- каждая строка: конкретный образ/действие/наблюдение (метро, подъезд, никотин, похмелье, холод, бессонница)

assistant
<VERSE speaker=alekhin>
Говорят, что нелепый человек на пороге жизни
Не видит перспектив, но я увидел
Прикоснулся к смерти, а она была мягкой как манго
И это было прекрасно, но теперь мне жалко себя
Когда умираешь, отбирают твоё перво

## 6) Merge LoRA в fp16 модель

Это удобно, если потом конвертировать в MLX или просто запускать одним каталогом.

In [ ]:
from peft import PeftModel
from transformers import AutoModelForCausalLM

MERGED_DIR = "qwen_makulatura_merged_fp16"

# Грузим базу уже в fp16 (без 4bit)
base_fp16 = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
)

peft_model = PeftModel.from_pretrained(base_fp16, OUTPUT_DIR)
merged = peft_model.merge_and_unload()

merged.save_pretrained(MERGED_DIR, safe_serialization=True)
tokenizer.save_pretrained(MERGED_DIR)

!ls -lah qwen_makulatura_merged_fp16

In [11]:
# сейв
from google.colab import files
files.download("qwen_makulatura_lora.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>